In [3]:
import cv2
from PIL import Image
from pydantic import BaseModel
import numpy as np
import pandas as pd
import os

import ultralytics

In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
model = ultralytics.YOLO(model='yolov8m-pose.pt')

In [5]:
results = model("D:\INSA/5IF\MediaEval_ClassificationTask\photoTest.JPG", save=True, device=0)


image 1/1 D:\INSA\5IF\MediaEval_ClassificationTask\photoTest.JPG: 640x448 1 person, 56.0ms
Speed: 3.0ms preprocess, 56.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 448)
Results saved to runs\pose\predict8


In [ ]:
np.concatenate((results[0].keypoints.numpy().data, results[0].keypoints.numpy().xyn), axis=2)

In [ ]:
keypoints = results[0].keypoints.numpy()
print(keypoints)

In [ ]:
keypoints.conf[0].T

In [ ]:
keyptsTotal = np.concatenate((keypoints.data, keypoints.xyn), axis=2)
print(keyptsTotal[0])

In [ ]:
kptsLine = np.reshape(keyptsTotal, (1,np.product(keyptsTotal.shape)))
print(kptsLine)

In [ ]:
boxe = results[0].boxes.numpy()
print(boxe)

In [ ]:
np.concatenate((results[0].boxes.numpy().conf, results[0].boxes.numpy().xywh[0], results[0].boxes.numpy().xywhn[0]), axis=0)

In [ ]:
boxeTotal = np.concatenate((boxe.conf, boxe.xywh[0], boxe.xywhn[0]), axis=0)
print(boxeTotal)

In [ ]:
total = np.concatenate((boxeTotal, kptsLine[0]), axis=0)

In [ ]:
print(total)

In [ ]:
liste = [total, total]
print(liste)

In [ ]:
df = pd.DataFrame(liste)
print(df)

In [ ]:

header = ["box_conf", "box_x", "box_y", "box_w", "box_h", "box_x_n", "box_y_n", "box_w_n", "box_h_n",
            "1_x", "1_y", "1_conf", "1_x_n", "1_y_n",
            "2_x", "2_y", "2_conf", "2_x_n", "2_y_n",
            "3_x", "3_y", "3_conf", "3_x_n", "3_y_n",
            "4_x", "4_y", "4_conf", "4_x_n", "4_y_n",
            "5_x", "5_y", "5_conf", "5_x_n", "5_y_n",
            "6_x", "6_y", "6_conf", "6_x_n", "6_y_n",
            "7_x", "7_y", "7_conf", "7_x_n", "7_y_n",
            "8_x", "8_y", "8_conf", "8_x_n", "8_y_n",
            "9_x", "9_y", "9_conf", "9x_n", "9_y_n",
            "10_x", "10_y", "10_conf", "10_x_n", "10_y_n",
            "11_x", "11_y", "11_conf", "11_x_n", "11_y_n",
            "12_x", "12_y", "12_conf", "12_x_n", "12_y_n",
            "13_x", "13_y", "13_conf", "13_x_n", "13_y_n",
            "14_x", "14_y", "14_conf", "14_x_n", "14_y_n",
            "15_x", "15_y", "15_conf", "15_x_n", "15_y_n",
            "16_x", "16_y", "16_conf", "16_x_n", "16_y_n",
            "17_x", "17_y", "17_conf", "17_x_n", "17_y_n",
        ]
df = df.set_axis(header,axis=1) # adding headers to the dataframe
df.to_csv("D:\INSA/5IF\MediaEval_ClassificationTask\photoTest.csv", sep=';', decimal='.', index=False) # store keypoint csv


In [ ]:
print(df)

In [ ]:
results2 = model("D:\INSA/5IF\MediaEval_ClassificationTask\photoTest2Boxes.jpeg", save=True)
boxe = results2[0].boxes.numpy()
print(boxe)

In [9]:
def analyse(video_path, csv_path):
    # Open the video file
    #video_path = "D:\INSA/5IF\MediaEval_ClassificationTask/videoTest.mp4"
    cap = cv2.VideoCapture(video_path)

    results = []
    # Loop through the video frames
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Run YOLOv8 inference on the frame
            result = model(frame)

            # Save relevent data
            kpts = np.concatenate((result[0].keypoints.numpy()[0].data, result[0].keypoints.numpy()[0].xyn), axis=2)
            kpts = np.reshape(kpts, (1, np.product(kpts.shape)))
            boxe = np.concatenate((result[0].boxes.numpy()[0].conf, result[0].boxes.numpy()[0].xywh[0], result[0].boxes.numpy()[0].xywhn[0]), axis=0)
            total = np.concatenate((boxe, kpts[0]), axis=0)

            results.append(total)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break

    df = pd.DataFrame(results)
    header = ["box_conf", "box_x", "box_y", "box_w", "box_h", "box_x_n", "box_y_n", "box_w_n", "box_h_n",
                "1_x", "1_y", "1_conf", "1_x_n", "1_y_n",
                "2_x", "2_y", "2_conf", "2_x_n", "2_y_n",
                "3_x", "3_y", "3_conf", "3_x_n", "3_y_n",
                "4_x", "4_y", "4_conf", "4_x_n", "4_y_n",
                "5_x", "5_y", "5_conf", "5_x_n", "5_y_n",
                "6_x", "6_y", "6_conf", "6_x_n", "6_y_n",
                "7_x", "7_y", "7_conf", "7_x_n", "7_y_n",
                "8_x", "8_y", "8_conf", "8_x_n", "8_y_n",
                "9_x", "9_y", "9_conf", "9x_n", "9_y_n",
                "10_x", "10_y", "10_conf", "10_x_n", "10_y_n",
                "11_x", "11_y", "11_conf", "11_x_n", "11_y_n",
                "12_x", "12_y", "12_conf", "12_x_n", "12_y_n",
                "13_x", "13_y", "13_conf", "13_x_n", "13_y_n",
                "14_x", "14_y", "14_conf", "14_x_n", "14_y_n",
                "15_x", "15_y", "15_conf", "15_x_n", "15_y_n",
                "16_x", "16_y", "16_conf", "16_x_n", "16_y_n",
                "17_x", "17_y", "17_conf", "17_x_n", "17_y_n",
            ]
    df = df.set_axis(header,axis=1) # adding headers to the dataframe
    df.to_csv(csv_path, sep=';', decimal='.', index=False) # store keypoint csv

    # Release the video capture object and close the display window
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
folder = "test"
folder != "test"

In [10]:
def analyseDataset(pathToData, pathToOutput):
    listeErreur = []

    for folder in sorted(os.listdir(pathToData)):
        if (not folder.startswith(".")) and os.path.isdir(os.path.join(pathToData, folder)):
            if folder != "test":
                pathToCSVFolder = os.path.join(pathToOutput, folder) # create a CSV folder for train and validation datasets
                if not os.path.exists(pathToCSVFolder):
                    os.mkdir(pathToCSVFolder)
                pathToFolder = os.path.join(pathToData, folder)
                for subFolder in sorted(os.listdir(pathToFolder)): 
                    if not subFolder.startswith("."):
                        pathToCSVSubFolder = os.path.join(pathToCSVFolder, subFolder) # create a CSV folder for each class of strokes
                        if not os.path.exists(pathToCSVSubFolder):
                            os.mkdir(pathToCSVSubFolder)
                        pathToSubFolder = os.path.join(pathToFolder, subFolder)
                        for file in sorted(os.listdir(pathToSubFolder)):
                            if (not file.startswith(".")) and (file.endswith(".mp4")):
                                pathToFile = os.path.join(pathToSubFolder, file)
                                CSVFileName = file[:len(file) - 4]
                                pathToCSV = os.path.join(pathToCSVSubFolder, CSVFileName+".csv")
                                if not os.path.exists(pathToCSV):
                                    print(pathToFile)
                                    try:
                                        analyse(pathToFile, pathToCSV)
                                    except:
                                        print("Erreur fichier: "+pathToFile)
                                        listeErreur.append(pathToFile)
    return listeErreur



In [ ]:
analyse("D:\INSA/5IF\MediaEval_ClassificationTask/testsVideos/testTrain/frontAtta/397579639.mp4", "D:\INSA/5IF\MediaEval_ClassificationTask/testCSV/test.csv")

In [11]:
analyseDataset("D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask", "D:\INSA/5IF\MediaEval_ClassificationTask\dataSetCSV")

D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Negative\276763674.mp4


0: 384x640 1 person, 236.6ms
Speed: 5.0ms preprocess, 236.6ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 48.5ms
Speed: 3.0ms preprocess, 48.5ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 49.1ms


Erreur fichier: D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Negative\276763674.mp4
D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Negative\528913764.mp4
Erreur fichier: D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Negative\528913764.mp4
D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Negative\5319642108.mp4


Speed: 2.1ms preprocess, 49.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 49.3ms
Speed: 2.0ms preprocess, 49.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 42.0ms


Erreur fichier: D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Negative\5319642108.mp4
D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Negative\54107653102.mp4
Erreur fichier: D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Negative\54107653102.mp4
D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Negative\55131103102.mp4


...


0: 384x640 1 person, 30.0ms
Speed: 2.0ms preprocess, 30.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 32.0ms
Speed: 2.0ms preprocess, 32.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)


D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Offensive Forehand Flip\838782121.mp4
Erreur fichier: D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Offensive Forehand Flip\838782121.mp4
D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Offensive Forehand Flip\893329121.mp4
Erreur fichier: D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Offensive Forehand Flip\893329121.mp4
D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Offensive Forehand Flip\91078531064.mp4



0: 384x640 1 person, 30.6ms
Speed: 2.0ms preprocess, 30.6ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 31.0ms
Speed: 2.0ms preprocess, 31.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 30.0ms
Speed: 2.5ms preprocess, 30.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)


Erreur fichier: D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Offensive Forehand Flip\91078531064.mp4
D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Offensive Forehand Flip\91762103510.mp4
Erreur fichier: D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Offensive Forehand Flip\91762103510.mp4
D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Offensive Forehand Flip\942769684.mp4
Erreur fichier: D:\INSA/5IF\MediaEval_ClassificationTask\data\classificationTask\train\Offensive Forehand Flip\942769684.mp4


KeyboardInterrupt: 

In [ ]:
for r in results:
    print(r.keypoints)

In [ ]:
model = ultralytics.YOLO(model='yolov8m-pose.pt')

In [ ]:
result = model('D:\INSA/5IF\MediaEval_ClassificationTask/314984131.mp4', conf=0.3, save=True)


In [ ]:
for r in result:
    print(r.boxes.xyxy)